In [11]:
import random
from IPython.display import clear_output

suits = ("Hearts","Diamonds","Spades","Clubs")
ranks = ("Two","Three","Four","Five","Six","Seven","Eight","Nine",
         "Ten","Jack","Queen","King","Ace")
values = {"Two":2,"Three":3,"Four":4,"Five":5,"Six":6,"Seven":7,"Eight":8,
          "Nine":9,"Ten":10,"Jack":10,"Queen":10,"King":10,"Ace":11}

In [2]:
class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        self.value = values[rank]
        
    def __str__(self):
        return self.rank + " of " + self.suit        

In [3]:
class Deck:
    def __init__(self):
        self.all_cards = []
        
        for suit in suits:
            for rank in ranks:
                created_card = Card(suit,rank)
                self.all_cards.append(created_card)
    
    def __len__(self):
        return len(self.all_cards)
    
    def shuffle(self):
        random.shuffle(self.all_cards)
        
    def deal_one(self):
        return self.all_cards.pop()
    
    def back_to_deck(self, cards):
        self.all_cards.extend(cards)

In [4]:
class Dealer:
    def __init__(self):
        self.hand = []
        
    def add_card(self, new_card):
        self.hand.append(new_card)
        
    def remove_all_cards(self):
        removed_cards = self.hand
        self.hand = []
        return removed_cards
    
    def display_hand_fully(self):
        result = "[" + str(self.hand[0])
        for i in range(1, len(self.hand)):
            result = result + ", " + str(self.hand[i])
        return result + "]"
    
    def display_hand_partially(self):
        result = "[" + str(self.hand[0])
        for i in range(1, len(self.hand)):
            if i == 1:
                result = result + ", ***"
            else:
                result = result + ", " + str(self.hand[i])
        return result + "]"
    
    def hand_value(self):
        sum = 0
        for card in self.hand:
            sum += card.value
        
        if sum > 21:
            for card in self.hand:
                if card.rank == "Ace":
                    sum -= 10
                    if sum <= 21:
                        break
    
        return sum

In [5]:
class Player(Dealer):
    def __init__(self, name, amount):
        self.name = name
        self.amount = amount
        self.hand = []
    
    def __str__(self):
        return f"Player {self.name} has £{self.amount} left to bet"
    
    def another_card(self):
        ask = " "
        while ask not in ["yes", "no", "y","n"]:
            ask = input("Would you like to hit? (Enter Yes or No) ").lower()
        if ask =="yes" or ask == "y":
            return True
        else:
            return False
    
    def winnings(self, number):
        self.amount = self.amount + number
        return f"You now have £{self.amount}"  
    
    def betting(self):
        bet = "a"
        count = 1

        while not bet.isdigit() or int(bet) > self.amount or self.amount < int(bet) * 5:
            clear_output()
            if count > 1 and not bet.isdigit():
                print("Please put a digit")
            if count > 1 and bet.isdigit() and self.amount < int(bet) * 5:
                print("You don't have the available funds")
            bet = input(f"How many chips would you like to bet (A chip is worth £5)? ")
            count +=1
            
        self.amount = self.amount - (5 * int(bet))
        print(f"You have £{self.amount} left")
        return bet


In [6]:
def play_again():
    ans = " "
    count = 0
    while ans not in ["yes","no","y","n"]:
        if count > 0:
            clear_output()
            print("I don't understand")
        ans = input("Would you like to play another hand? (Enter Yes or No) ").lower()
    if ans == "yes" or ans =="y":
        return True
    else:
        return False

In [9]:
# GAME SETUP
new_deck = Deck()
new_dealer = Dealer()
player_one = Player("Player One", 100)

In [10]:
game_on = True

while game_on:
    player_bust = False
    dealer_bust = False
    hit = True
    new_deck.shuffle()
    bet = player_one.betting()
    #giving out hands
    for i in range(0,2):
        player_one.add_card(new_deck.deal_one())
        new_dealer.add_card(new_deck.deal_one())
    
   #Dealing 
    
    while player_one.hand_value() <= 21 and hit == True:
        print("Dealer's hand is: " + new_dealer.display_hand_partially())
        print("Your hand is: " + player_one.display_hand_fully())
        print(f"The value of your hand is {player_one.hand_value()}")
        if player_one.another_card():
            player_one.add_card(new_deck.deal_one())
        else:
            hit = False
        if player_one.hand_value() > 21:
            print("Your hand is: " + player_one.display_hand_fully())
            print(f"The value of your hand is {player_one.hand_value()}")
            print("You have gone bust")
            player_bust = True
            
    while new_dealer.hand_value() <= player_one.hand_value() and new_dealer.hand_value() < 17 and player_bust == False:
        new_dealer.add_card(new_deck.deal_one())
        if new_dealer.hand_value() > 21:
            print("Dealer went bust!")
            dealer_bust = True
    #printing out hands. if statement for player bust cause otherwise repeated print satement        
    if player_bust == False:
        print(f"Your hand is {player_one.display_hand_fully()} and the value is {player_one.hand_value()}")
    print(f"The dealer's hand is {new_dealer.display_hand_fully()} and the value is {new_dealer.hand_value()}")
    
    # Who wins?
    if (player_bust == False and player_one.hand_value() > new_dealer.hand_value()) or (player_bust == False and dealer_bust == True):
        print(f"Congratulations, you win! You won {int(bet) * 2} chips")
        player_one.winnings(int(bet) * 2 * 5)
        
    elif (player_bust == dealer_bust == False) and (player_one.hand_value() == new_dealer.hand_value()):
        print("Draw! You got back your " + bet + " chips")
        player_one.winnings(int(bet) * 5)
    else:
        print("The dealer won! You lost " + bet + " chips")
        
    #printing out amount left and return cards to deck
    print(f"The amount you have left is £{player_one.amount}")
    new_deck.back_to_deck(player_one.remove_all_cards())
    new_deck.back_to_deck(new_dealer.remove_all_cards())
    
    again = play_again()
    if not again:
        game_on = False
    clear_output()